In [3]:
import sys
import os

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

from run import main

# Finds thresholds for IH-confidence and IH-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/hosp_dengue",
    train_name="hospitalization_dengue_23",
    measure="ih",  # or "if"
    metric="macro-f1",
    steps=[#"build", 
           #"results", 
           #"reports", "plot", 
           "grid", "heuristic", #"brute", "annealing" 
          ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.9
Rejecting threshold(s): 0.94
Associated minimum cost: 0.528289

Uncertainty-based rejection
Filtering threshold(s): 0.625
Rejecting threshold(s): 0.5
Associated minimum cost: 0.881839


In [4]:
# Finds thresholds for IF-confidence and IF-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/hosp_dengue",
    train_name="hospitalization_dengue_23",
    measure="if",  # or "if"
    metric="macro-f1",
    steps=[#"build",
           #"results", "reports", "plot", 
        "grid", "heuristic",#"brute", "annealing" 
    ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.95
Rejecting threshold(s): 0.82
Associated minimum cost: 0.563013

Uncertainty-based rejection
Filtering threshold(s): 0.975
Rejecting threshold(s): 0.5
Associated minimum cost: 0.908716


In [1]:
from thresholds_tester import ThresholdsTester

import sys
import os

# Test the thresholds found in the test set

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

tester = ThresholdsTester(
    data_path="(...)/RefineAI/experiments/hosp_dengue",
    train_name="hospitalization_dengue_23",
    test_name="hospitalization_dengue_24",
    metric="macro-f1",
    ih_uncertainty=[0.625, 0.5], #filtering, rejecting
    ih_confidence=[0.9, 0.94],
    if_uncertainty=[0.975, 0.5],
    if_confidence=[0.95, 0.82]
)

tester.final_measures()
final, macro = tester.final_results()

In [3]:
# Reorganizes the results 

final = final.reindex([2, 1, 0, 5, 4, 3, 8, 7, 6, 11, 10, 9, 12, 13]) 
final

,measure,metric,T_value,thresholds,precision_class_0,precision_class_1,recall_class_0,recall_class_1,rate_accep_0,rate_accep_1,mean_conf_uncert,f1_0,f1_1,f1-macro,acp_a
2,ih,C,0.000,0.94,0.951,0.843,0.721,0.976,0.250,0.583,0.955,0.820181,0.904638,0.862409,0.4165
1,ih,C,0.100,0.94,0.909,0.714,0.732,0.901,0.797,0.901,0.988,0.810954,0.796674,0.803814,0.8490
0,ih,C,0.100,0.00,0.884,0.656,0.704,0.860,1.000,1.000,0.955,0.783798,0.744274,0.764036,1.0000
5,ih,U,0.000,0.50,0.882,0.665,0.716,0.855,1.000,1.000,0.603,0.790378,0.748125,0.769251,1.0000
4,ih,U,0.375,0.50,0.856,0.672,0.739,0.811,0.999,1.000,0.698,0.793209,0.734986,0.764097,0.9995
3,ih,U,0.375,0.00,0.856,0.671,0.739,0.811,1.000,1.000,0.698,0.793209,0.734387,0.763798,1.0000
8,if,C,0.000,0.82,0.919,0.744,0.755,0.914,0.703,0.832,0.927,0.828967,0.820285,0.824626,0.7675
7,if,C,0.050,0.82,0.917,0.741,0.757,0.910,0.721,0.837,0.929,0.829354,0.816850,0.823102,0.7790
6,if,C,0.050,0.00,0.880,0.665,0.719,0.851,1.000,1.000,0.871,0.791395,0.746590,0.768992,1.0000
11,if,U,0.000,0.50,0.882,0.665,0.716,0.855,1.000,1.000,0.603,0.790378,0.748125,0.769251,1.0000


In [4]:
# Calculates the cost for each solution, used to statistical comparison

macro = macro[['measure', 'metric', 'f1-macro','acp_a','mean_conf_uncert']]
macro = round(macro,3)

macro['cost'] = 4*(1-macro['f1-macro']) + (1-(macro['acp_a'])) + (1-macro['mean_conf_uncert'])

macro_confidence = macro.reindex([1, 7, 12])
macro_uncertainty = macro.reindex([4, 10, 13])

print(', '.join([f"'{x:.3}'" for x in macro_uncertainty.iloc[:,-1]]))

'1.25', '1.33', '1.32'


In [5]:
print(', '.join([f"'{x:.3}'" for x in macro_confidence.iloc[:,-1]]))

'0.947', '1.0', '1.06'
